In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib import request
import re
import json
from tqdm import tqdm
import datetime

In [2]:
data = pd.read_csv("C:/Users/dhxog/Documents/카카오톡 받은 파일/youtube_merge_final.csv")


In [3]:
production = data["production"]
productionname = list()
for i in np.arange(len(production)):
    productionname.append(production[i])
temp = set(productionname)
temp = list(temp)

In [4]:
temp.remove('Def Jam Recordings, a division of UMG Recordings, Inc')

In [5]:
production = list()
for i in np.arange(len(temp)):
    production = production + temp[i].split(",")
m = re.compile(".+(?=\()")
productionname = list()
for i in np.arange(len(production)):
    t = m.search(production[i])
    if t == None:
        productionname.append(production[i])
    else:
        productionname.append(t.group())
production = list()
for i in np.arange(len(productionname)):
    production.append(productionname[i].strip())
temp = set(production)
production = list(temp)

In [6]:
client_id = "RLpoOMH7S702D1xpvVfA"
client_secret = "RbICRLuPbD"
url = "https://openapi.naver.com/v1/datalab/search";

In [7]:
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")

In [8]:
keywords = production

In [9]:
productiontrend = pd.DataFrame()
indexlist = list()
for i in tqdm(np.arange(0, len(keywords))):

    bodytext = {"startDate":"2018-04-22",
            "endDate":"2020-04-25",
            "timeUnit":"week",
            "keywordGroups":[{"groupName":"{}".format(keywords[i]),
                              "keywords": ["{}".format(keywords[i])]}]
                             }

    body = json.dumps(bodytext,ensure_ascii = False )
    response = urllib.request.urlopen(request, data=body.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        scraped = response_body.decode('utf-8')
    else:
            print("Error Code:" + rescode)
    result = json.loads(scraped)
    data = result["results"][0]["data"]
    time = list(i["period"] for i in data)
    value = pd.DataFrame(i["ratio"] for i in data)
    
    # 실행이 되는 index만 뽑아준다
    if str(value) != 'Empty DataFrame\nColumns: []\nIndex: []':
        indexlist = indexlist + [keywords[i]]
    
    
    productiontrend = productiontrend.append(np.transpose(value))

100%|██████████| 296/296 [00:46<00:00,  6.38it/s]


In [10]:
productiontrend.columns = time
productiontrend.index = indexlist

In [16]:
productiontrend = np.transpose(productiontrend.fillna(0))

In [17]:
productiontrend.to_csv("소속사trend크롤링.csv", mode = 'w', encoding = 'utf-8-sig')